**Pre-Processing**

In [1]:
#Install kaggle
!pip install -q kaggle

In [ ]:
#upload the  kaggle json file
from google.colab import files
files.upload()

In [3]:
# Create a directory named "kaggle"
!mkdir ~/.kaggle

In [4]:
#Copy the kaggle.json to created folder
! cp kaggle.json ~/.kaggle/

In [5]:
#permisson for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#upload here api command that taken from the dataset
!kaggle datasets download -d aayushsoni4/tmdb-5000-movie-dataset-with-ratings

In [ ]:
#unzip the file
!unzip tmdb-5000-movie-dataset-with-ratings.zip

In [8]:
import pandas as pd
credits=pd.read_csv('/content/tmdb_movie_credits.csv')
dataset=pd.read_csv('/content/tmdb_movie_dataset.csv')
rating =pd.read_csv('/content/tmdb_movie_ratings.csv')

In [ ]:
movie=dataset.merge(credits,on='title')
movie.info()

In [ ]:
# Select specific columns
movie=movie[['title','genres', 'keywords', 'original_language', 'overview', 'cast', 'crew']]

# Display the first row
movie.head(1)

In [ ]:
movie.isnull().sum() #before dropping null value

In [12]:
# movie.dropna(inplace=True)  #in overview section only 1 value is null so I droped it

In [13]:
# movie.isnull().sum() #after droping null value

In [ ]:
movie.duplicated().sum() #checking the duplicated row

In [ ]:
movie.iloc[1].genres #invide movie with row 0 we are accessing only genres value

In [16]:
import ast  # helps in converting string to dictionary or list format

def convert1(obj):  # function
    fetch = []  # empty list
    try:
        for i in ast.literal_eval(obj):     # accessing the value
            fetch.append(i['name'])
    except ValueError:
        print("Error: Input is not a valid Python expression.")
    return fetch  # returning the list

In [17]:
movie['genres'] = movie['genres'].apply(convert1)

In [ ]:
movie['keywords'][0]

In [19]:
movie['keywords'] = movie['keywords'].apply(convert1)

In [20]:
def convert3(obj):  # function
    fetch = []  # empty list
    counter=0
    try:
        for i in ast.literal_eval(obj):
          if counter!=3:    # accessing the value
            fetch.append(i['name'])
          else:
            break
    except ValueError:
        print("Error: Input is not a valid Python expression.")
    return fetch  # returning the list

In [21]:
movie['cast'] = movie['cast'].apply(convert3)

In [22]:
def convert(obj):  # function
    fetch = []  # empty list
    try:
        for i in ast.literal_eval(obj):    # accessing the value
          if i['job']=='Director':
            fetch.append(i['name'])
            break
    except ValueError:
        print("Error: Input is not a valid Python expression.")
    return fetch  # returning the list

In [23]:
movie['crew'] = movie['crew'].apply(convert3)

In [ ]:
movie['overview'][1]  #Before split

In [ ]:
movie.head(1)

In [26]:
# Applying lambda function to split each overview text into a list of words
movie['overview']= movie['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

In [ ]:
#Removing all the spaces
movie['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movie['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movie['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movie['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [28]:
#Concating genres, keywords , cast ,crew
movie['tags']=movie['genres']+movie['keywords']+movie['cast']+movie['crew']

In [29]:
movie_df=movie[['title','tags']]  #creating new dataframe with title and tags

In [ ]:
#converting list of tags into string format
movie_df['tags']=movie_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
movie_df['tags']=movie_df['tags'].apply(lambda x:x.lower())

In [ ]:
movie_df.sample(5)

**Text Visulization**

The Porter Stemmer is a stemming algorithm, specifically designed to transform words into their root forms. It's part of the Natural Language Toolkit (NLTK), which is a Python library commonly used for natural language processing tasks.

In [33]:
import nltk # eg. love,loves,loved conveted into it's root form as love
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [34]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [ ]:
movie_df['tags']=movie_df['tags'].apply(stem)

In [36]:
#Convert a collection of text documents to a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer( max_features=5000,stop_words='english')  #here stop words means it not count the stop words like to, in from, by etc
vector=cv.fit_transform(movie_df['tags']).toarray()

In [ ]:
vector #converting most frequently tags used in movie into binary formate inside the array

In [ ]:
# checking the first 100 frequent counts in inside the 5000 list
empty=[]
for i in range(0,100):
  empty.append(cv.get_feature_names_out()[i]) #these are the 5000 words that are mostly repeated in the movie dataset
empty


In [ ]:
 #this will give the list of stop words
cv.get_stop_words()

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [ ]:
#This will help in giving the index of the movie name
movie_df[movie_df['title']=='Four Rooms'].index[0]
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[0])[1:6]

In [42]:
def recommend(movie):
  index_value=movie_df[movie_df['title']==movie].index[0]
  distance=similarity[index_value]
  movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movie_list:
    print(movie_df.iloc[i[0]].title)


In [51]:
import pickle

In [62]:
pickle.dump(movie_df.to_dict(),open('movie_list.pkl','wb'))

In [63]:
pickle.dump(similarity,open('similarity.pkl','wb'))